In [ ]:
Dado el código anterior, modificar el codigo de la intervención para que use el patron decorator.

In [6]:
from abc import ABC, abstractmethod
from datetime import date

# Clases de Vehículo (sin cambios)
class Vehiculo(ABC):
    def __init__(self, marca, modelo, matricula):
        self.marca = marca
        self.modelo = modelo
        self.matricula = matricula

    @abstractmethod
    def get_tipo(self):
        pass

class Coche(Vehiculo):
    def get_tipo(self):
        return "Coche"

class Moto(Vehiculo):
    def get_tipo(self):
        return "Moto"

class Furgoneta(Vehiculo):
    def get_tipo(self):
        return "Furgoneta"

class Autobus(Vehiculo):
    def get_tipo(self):
        return "Autobus"

# Factory para Vehículos (sin cambios)
class VehiculoFactory:
    @staticmethod
    def crear_vehiculo(tipo, marca, modelo, matricula):
        tipos = {
            "coche": Coche,
            "moto": Moto,
            "furgoneta": Furgoneta,
            "autobus": Autobus
        }
        return tipos.get(tipo.lower())(marca, modelo, matricula)

# Clase Cliente (sin cambios)
class Cliente:
    def __init__(self, nombre, apellido, dni):
        self.nombre = nombre
        self.apellido = apellido
        self.dni = dni
        self.vehiculos = []

    def añadir_vehiculo(self, vehiculo):
        self.vehiculos.append(vehiculo)

# Mixin para el cálculo del coste
class CosteMixin:
    def calcular_coste(self, horas, precio_hora, componentes):
        return horas * precio_hora + componentes
# Implementación del patrón Decorator para Servicios
class ServicioComponent(ABC):
    @abstractmethod
    def get_coste(self):
        pass

    @abstractmethod
    def get_descripcion(self):
        pass

class ServicioBase(ServicioComponent):
    def __init__(self, nombre, descripcion, precio_hora, horas):
        self.nombre = nombre
        self.descripcion = descripcion
        self.precio_hora = precio_hora
        self.horas = horas

    def get_coste(self):
        return self.precio_hora * self.horas

    def get_descripcion(self):
        return f"{self.nombre}: {self.descripcion}"

class ServicioDecorator(ServicioComponent):
    def __init__(self, servicio):
        self._servicio = servicio

    def get_coste(self):
        return self._servicio.get_coste()

    def get_descripcion(self):
        return self._servicio.get_descripcion()

class ComponenteAdicional(ServicioDecorator):
    def __init__(self, servicio, nombre, coste):
        super().__init__(servicio)
        self.nombre = nombre
        self.coste = coste

    def get_coste(self):
        return super().get_coste() + self.coste

    def get_descripcion(self):
        return f"{super().get_descripcion()} + {self.nombre}"

# Clase Intervención (modificada para usar ServicioComponent)
class Intervencion:
    def __init__(self, cliente, fecha):
        self.cliente = cliente
        self.fecha = fecha
        self.vehiculos = []
        self.servicios = []

    def añadir_vehiculo(self, vehiculo):
        self.vehiculos.append(vehiculo)

    def añadir_servicio(self, servicio):
        self.servicios.append(servicio)

# Clase Factura (modificada para usar ServicioComponent)
class Factura:
    def __init__(self, intervencion):
        self.intervencion = intervencion

    def calcular_total(self):
        return sum(servicio.get_coste() for servicio in self.intervencion.servicios)

    def imprimir(self):
        print(f"Factura para {self.intervencion.cliente.nombre} {self.intervencion.cliente.apellido}")
        print(f"Fecha: {self.intervencion.fecha}")
        print("Vehículos:")
        for vehiculo in self.intervencion.vehiculos:
            print(f"- {vehiculo.get_tipo()}: {vehiculo.marca} {vehiculo.modelo} ({vehiculo.matricula})")
        print("Servicios:")
        for servicio in self.intervencion.servicios:
            print(f"- {servicio.get_descripcion()}: {servicio.get_coste()}€")
        print(f"Total: {self.calcular_total()}€")

# Ejemplo de uso
if __name__ == "__main__":
    # Crear fábrica de vehículos
    factory = VehiculoFactory()

    # Crear vehículos
    coche = factory.crear_vehiculo("coche", "Toyota", "Corolla", "1234ABC")
    moto = factory.crear_vehiculo("moto", "Honda", "CBR", "5678XYZ")

    # Crear cliente
    cliente = Cliente("Juan", "Pérez", "12345678A")
    cliente.añadir_vehiculo(coche)
    cliente.añadir_vehiculo(moto)
    cambio_aceite = ServicioBase("Cambio de aceite", "Cambio de aceite y filtro", 40, 1.5)
    cambio_aceite_con_filtro_premium = ComponenteAdicional(cambio_aceite, "Filtro de aceite premium", 20)

    revision_frenos = ServicioBase("Revisión de frenos", "Comprobación y ajuste de frenos", 50, 2)
    revision_frenos_con_pastillas = ComponenteAdicional(revision_frenos, "Cambio de pastillas de freno", 60)

    revision_electrica = ServicioBase("Revisión de componentes electronicos", "Comprobación y ajuste de componentes electronicos", 50, 4)
    revision_electrica_substitucion_faros_antiniebla = ComponenteAdicional(revision_electrica, "Subsitución de faros antiniebla", 500)
    pulido_faros = ComponenteAdicional(revision_electrica_substitucion_faros_antiniebla, "Pulido Faros", 100)



    intervencion = Intervencion(cliente, date.today())
    intervencion.añadir_vehiculo(cliente.vehiculos[0])
    intervencion.añadir_servicio(cambio_aceite_con_filtro_premium)
    intervencion.añadir_servicio(revision_frenos_con_pastillas)
    intervencion.añadir_servicio(pulido_faros)

    # Crear y mostrar factura
    factura = Factura(intervencion)
    factura.imprimir()



Factura para Juan Pérez
Fecha: 2024-08-01
Vehículos:
- Coche: Toyota Corolla (1234ABC)
Servicios:
- Cambio de aceite: Cambio de aceite y filtro + Filtro de aceite premium: 80.0€
- Revisión de frenos: Comprobación y ajuste de frenos + Cambio de pastillas de freno: 160€
- Revisión de componentes electronicos: Comprobación y ajuste de componentes electronicos + Subsitución de faros antiniebla + Pulido Faros: 800€
Total: 1040.0€
